In [ ]:
!pip install tensorflow-text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import pandas as pd

df = pd.read_csv("/content/main.csv")

In [ ]:
df.head(5)

,label,title
0,non-clickbait,"Masuk Radar Pilwalkot Medan, Menantu Jokowi Be..."
1,non-clickbait,Malaysia Sudutkan RI: Isu Kabut Asap hingga In...
2,clickbait,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...
3,non-clickbait,"Kemensos Salurkan Rp 7,3 M bagi Korban Kerusuh..."
4,non-clickbait,"Terkait Mayat Bayi Mengenaskan di Tangerang, S..."


In [ ]:
df.groupby("label").describe()

title  ...     
              count  ... freq
label                ...     
clickbait      6290  ...    2
non-clickbait  8710  ...    2

[2 rows x 4 columns]

In [ ]:
df_cb = df[df["label"] == "clickbait"]
df_cb.shape

(6290, 2)

In [ ]:
df_ncb = df[df["label"] == "non-clickbait"]
df_ncb.shape

(8710, 2)

In [ ]:
df_ncb_downsampled = df_ncb.sample(df_cb.shape[0])
df_ncb_downsampled.shape

(6290, 2)

In [ ]:
df_balanced = pd.concat([df_cb, df_ncb_downsampled])

In [ ]:
df_balanced["label"].value_counts()

non-clickbait    6290
clickbait        6290
Name: label, dtype: int64

In [ ]:
df_balanced["clickbait"] = df_balanced["label"].apply(lambda x:1 if x=="clickbait" else 0)

In [ ]:
df_balanced.head()

,label,title,clickbait
2,clickbait,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,1
11,clickbait,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",1
13,clickbait,Pesan Gamblang Poyuono Menolak Revisi UU KPK,1
37,clickbait,Detik-detik Lima Kendaraan Alami Kecelakaan Be...,1
38,clickbait,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,1


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['title'],df_balanced['clickbait'], stratify=df_balanced['clickbait'])

In [ ]:
X_train.head(4)

10176    Ribuan Rumah Bakal Tergusur Proyek Double Trac...
9528     Menjajal Jaringan Ngebut Smartfren di Kawasan ...
7264     JK : Sound System Masjid kadang Mengganggu, Ak...
13962    Laudya Cynthia Bella Mewek Tanpa Cincin Kawin ...
Name: title, dtype: object

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "jokowi presiden kita", 
    "apa kabar kasus ahok?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84585255, -0.30646417, -0.5881597 , ..., -0.35923603,
        -0.5986587 ,  0.8222759 ],
       [-0.85944283, -0.502674  , -0.9666674 , ..., -0.91528577,
        -0.71611214,  0.8117752 ]], dtype=float32)>

In [ ]:
coba = get_sentence_embeding(["jokowi", "prabowo","indonesia", 
                              "medan", "madura", "pisang"])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([coba[0]],[coba[1]])

array([[0.981042]], dtype=float32)

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
len(X_train)

9435

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
295/295 [==============================] - 199s 639ms/step - loss: 0.6731 - accuracy: 0.5793 - precision: 0.5832 - recall: 0.5559
Epoch 2/10
295/295 [==============================] - 188s 639ms/step - loss: 0.6551 - accuracy: 0.5994 - precision: 0.6044 - recall: 0.5752
Epoch 3/10
295/295 [==============================] - 188s 637ms/step - loss: 0.6483 - accuracy: 0.6103 - precision: 0.6161 - recall: 0.5849
Epoch 4/10
295/295 [==============================] - 188s 636ms/step - loss: 0.6422 - accuracy: 0.6171 - precision: 0.6228 - recall: 0.5934
Epoch 5/10
295/295 [==============================] - 187s 634ms/step - loss: 0.6383 - accuracy: 0.6226 - precision: 0.6290 - recall: 0.5976
Epoch 6/10
295/295 [==============================] - 188s 636ms/step - loss: 0.6367 - accuracy: 0.6332 - precision: 0.6399 - recall: 0.6089
Epoch 7/10
295/295 [==============================] - 186s 631ms/step - loss: 0.6305 - accuracy: 0.6362 - precision: 0.6438 - recall: 0.6099
Epoch 8/10
29

In [ ]:
model.evaluate(X_test, y_test)

99/99 [==============================] - 62s 615ms/step - loss: 0.6149 - accuracy: 0.6696 - precision: 0.7401 - recall: 0.5232


[0.6149373054504395,
 0.6696343421936035,
 0.7401078939437866,
 0.5232040882110596]